In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

In [3]:
# Load dataset
#vegis_df = pd.read_csv("G:/Meine Ablage/KI_Projekt/Daten/vegis_traindata.csv")
#vegis_df = vegis_df.drop(columns=["Unnamed: 0", "green pixels", "Image"])
#print(vegis_df.head())
#print(vegis_df.shape)
X, y = load_iris(return_X_y=True)
print(X[:5])
print(y)
print("Shape X: ", X.shape)
print("Shape y: ", y.shape)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
Shape X:  (150, 4)
Shape y:  (150,)


In [4]:
df = pd.DataFrame(X)
df['4'] = y
df.head()
dataset = np.array(df)

In [5]:
dataset

array([[5.1, 3.5, 1.4, 0.2, 0. ],
       [4.9, 3. , 1.4, 0.2, 0. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [4.6, 3.1, 1.5, 0.2, 0. ],
       [5. , 3.6, 1.4, 0.2, 0. ],
       [5.4, 3.9, 1.7, 0.4, 0. ],
       [4.6, 3.4, 1.4, 0.3, 0. ],
       [5. , 3.4, 1.5, 0.2, 0. ],
       [4.4, 2.9, 1.4, 0.2, 0. ],
       [4.9, 3.1, 1.5, 0.1, 0. ],
       [5.4, 3.7, 1.5, 0.2, 0. ],
       [4.8, 3.4, 1.6, 0.2, 0. ],
       [4.8, 3. , 1.4, 0.1, 0. ],
       [4.3, 3. , 1.1, 0.1, 0. ],
       [5.8, 4. , 1.2, 0.2, 0. ],
       [5.7, 4.4, 1.5, 0.4, 0. ],
       [5.4, 3.9, 1.3, 0.4, 0. ],
       [5.1, 3.5, 1.4, 0.3, 0. ],
       [5.7, 3.8, 1.7, 0.3, 0. ],
       [5.1, 3.8, 1.5, 0.3, 0. ],
       [5.4, 3.4, 1.7, 0.2, 0. ],
       [5.1, 3.7, 1.5, 0.4, 0. ],
       [4.6, 3.6, 1. , 0.2, 0. ],
       [5.1, 3.3, 1.7, 0.5, 0. ],
       [4.8, 3.4, 1.9, 0.2, 0. ],
       [5. , 3. , 1.6, 0.2, 0. ],
       [5. , 3.4, 1.6, 0.4, 0. ],
       [5.2, 3.5, 1.5, 0.2, 0. ],
       [5.2, 3.4, 1.4, 0.2, 0. ],
       [4.7, 3

In [17]:
class DecissionTree:
    def __init__(self, max_depth, min_size):
        self.max_depth = max_depth
        self.min_size = min_size
        self.depth = 0
        self.classes_dataset = None

    def fit(self, dataset):
        """
        X: array(n,d)
        y: array(labels)
        """
        #self.classes_dataset = list(set(dataset[:,-1]))
        
        # Nach jedem Split gibt es ein neuen Datensatz der dann übergeben werden muss 
        root = self._get_split(dataset)
        self.depth += 1
        self._split(root)
        return root

    def _split(self, node):
        left, right = node['buckets']
        del(node['buckets'])
        #print("Left: ", left)
        #print("Right: ", right)
        # Check if there was no split. The lists are empty
        if not left or not right:
            # Add amount of each class to the node
            node['left'] = node['right'] = self._to_terminal(left + right)
            return
        if self.depth >= self.max_depth:
            node['left'], node['right'] = self._to_terminal(left), self._to_terminal(right)
            return
        if len(left) <= self.min_size:
            node['left'] = self._to_terminal(left)
        else:
            node['left'] = self._get_split(left)
            self.depth += 1
            self._split(node['left'])
        
        if len(right) <= self.min_size:
            node['right'] = self._to_terminal(right)
        else:
            node['right'] = self._get_split(right)
            self.depth += 1
            self._split(node['right'])
        
    def _get_split(self, dataset):
        best_feat, best_thresh, best_gini_loss, best_buckets = 999, 999, 999, None
        #unique_cls_values = list(set(dataset[:,-1]))
        print("DATASET: ,", dataset)
        unique_cls_values = list(set(row[-1] for row in dataset))
        # Do the split for each feature in the dataset (X)
        for idx in range(dataset.shape[1]-1): # do not use the label for split
            # assign all instances to a split
            print("Split for ", idx)
            for row in dataset:
                buckets = self._create_split(idx, row[idx], dataset)
            # evaluate the split for the two groups
            gini_loss = self._gini_loss(buckets, unique_cls_values)
            print("New Gini loss: ", gini_loss)

            # check if the split is better than the current best split. If true, keep it
            if gini_loss < best_gini_loss:
                best_gini_loss = gini_loss
                best_feat = idx
                best_thresh = row[idx]
                best_buckets = buckets

                #print(f"best gini: {best_gini_loss}\nbest feat Idx: {best_feat}\nbest thresh: {best_thresh}\nbest buckets: {best_buckets}")
        return {'feature_idx': best_feat, 'thresh': best_thresh, 'buckets': best_buckets}

    def _create_split(self, idx, thresh, dataset):
        # idx = current feature for the split
        # thresh = current threshold for the split. Now check value for the whole dataset. Then grab the next threshold
        left_bucket, right_bucket = list(), list()
        for row in dataset:
            if row[idx] <= thresh:
                left_bucket.append(int(row[-1]))
            else:
                right_bucket.append(int(row[-1]))

        return left_bucket, right_bucket

    def _gini_loss(self, buckets, cls_values):
        # cls_values: Are the values in the current dataset 
        print("Class Values i ngini : ", cls_values)
        n_instances = float(sum(len(bucket) for bucket in buckets))
        print("Number Instances: ", n_instances)

        gini = 0.0
        for bucket in buckets:
            size = len(bucket)
            # calculate gini score for left and right bucket
            if size == 0:
                continue
            score = 0.0
            for cls_val in cls_values:
                ratio = bucket.count(cls_val) / size
                score += ratio**2

            gini += (size / n_instances) * (1.0 - score)
        return gini
    
    def _to_leaf(self, bucket):
        pred_cls = {cls:0 for cls in self.classes_dataset}
        for c in self.classes_dataset:
            pred_cls[c] = bucket.count(c)
        return pred_cls
    
    def _to_terminal(self, bucket):
        outcomes = [row[0] for row in bucket]
        return max(set(outcomes), key=outcomes.count)
    


tree = DecissionTree(3, 1)

In [13]:
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
dataset_np = np.array(dataset)
dataset_np.shape

(10, 3)

In [18]:
t = tree.fit(dataset_np)

DATASET: , [[ 2.77124472  1.78478393  0.        ]
 [ 1.72857131  1.16976141  0.        ]
 [ 3.67831985  2.81281357  0.        ]
 [ 3.96104336  2.61995032  0.        ]
 [ 2.99920892  2.20901421  0.        ]
 [ 7.49754587  3.16295355  1.        ]
 [ 9.00220326  3.33904719  1.        ]
 [ 7.44454233  0.47668338  1.        ]
 [10.12493903  3.23455098  1.        ]
 [ 6.64228735  3.31998376  1.        ]]
Split for  0
Class Values i ngini :  [0.0, 1.0]
Number Instances:  10.0
New Gini loss:  0.1666666666666666
Split for  1
Class Values i ngini :  [0.0, 1.0]
Number Instances:  10.0
New Gini loss:  0.4444444444444444
DATASET: , [0, 0, 0, 0, 0, 1]


TypeError: 'int' object is not subscriptable